<a href="https://colab.research.google.com/github/candanbayar/2022352_ID_Wordle_Twitter_Dataset_/blob/main/Big_Data_Wordle_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages

In [1]:
# pyspark 

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=1e7da31c477eb1b37450d6bc9edef8793776e73bcc442b1c8196ccfe7123db7f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
!pip install -U textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import Libraries

In [3]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt   
from itertools import cycle
import re 

# import libraries about NLP 
import warnings
warnings.simplefilter("ignore")

import re
import numpy as np 
import pandas as pd
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

import gensim  
import nltk 
from nltk.sentiment import SentimentIntensityAnalyzer 
from textblob import TextBlob 
from wordcloud import WordCloud, STOPWORDS 

from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer
from nltk.tokenize.treebank import TreebankWordDetokenizer

import tensorflow as tf
import plotly.express as px 

# Modelling 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

# import keras 

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [4]:
# import pyspark

from pyspark.sql import SparkSession

In [5]:
# create variable called spark 

spark = SparkSession.builder.appName('wordle_tweet').getOrCreate()

In [6]:
# spark 

spark

In [7]:
tw = spark.read.csv("tweets_wordle.csv")

In [8]:
tw

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

In [9]:
# read the dataset 

spark.read.option('header','true').csv('tweets_wordle.csv').show()

+-----------+-------------------+--------------------+--------------+--------------+
|  wordle_id|           tweet_id|          tweet_date|tweet_username|    tweet_text|
+-----------+-------------------+--------------------+--------------+--------------+
|        210|1482553374591660037|2022-01-16 03:20:...|      bpszebes|Wordle 210 4/6|
|    ⬛⬛🟨🟨⬛|               null|                null|          null|          null|
|     🟩⬛⬛⬛⬛|               null|                null|          null|          null|
|   🟩🟩🟨⬛⬛|               null|                null|          null|          null|
|🟩🟩🟩🟩🟩"|               null|                null|          null|          null|
|        210|1482553387937898499|2022-01-16 03:20:...|    cruisecoup|Wordle 210 4/6|
|      ⬜⬜⬜⬜⬜|               null|                null|          null|          null|
|  🟩🟩🟨🟨⬜|               null|                null|          null|          null|
|  🟩🟩⬜🟩🟨|               null|                null|          null|          null|

In [10]:
# type dataset 
type(tw)

pyspark.sql.dataframe.DataFrame

In [11]:
# check the schema 

tw.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [15]:
# read dataset 

tw = pd.read_csv("tweets_wordle.csv")

In [16]:
tw.dtypes

wordle_id          int64
tweet_id           int64
tweet_date        object
tweet_username    object
tweet_text        object
dtype: object

In [17]:
tw.head()

,wordle_id,tweet_id,tweet_date,tweet_username,tweet_text
0,210,1482553374591660037,2022-01-16 03:20:43+00:00,bpszebes,Wordle 210 4/6\n\n⬛⬛🟨🟨⬛\n🟩⬛⬛⬛⬛\n🟩🟩🟨⬛⬛\n🟩🟩🟩🟩🟩
1,210,1482553387937898499,2022-01-16 03:20:46+00:00,cruisecoup,Wordle 210 4/6\n\n⬜⬜⬜⬜⬜\n🟩🟩🟨🟨⬜\n🟩🟩⬜🟩🟨\n🟩🟩🟩🟩🟩
2,210,1482553422276698113,2022-01-16 03:20:55+00:00,DestroVega,Wordle 210 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩⬜🟨⬜\n⬜🟩⬜🟨⬜\n🟩🟩🟩🟩🟩
3,210,1482553436910628866,2022-01-16 03:20:58+00:00,brenmardash,Wordle 210 3/6\n\n⬜⬜🟨⬜⬜\n🟨🟨⬜⬜⬜\n🟩🟩🟩🟩🟩
4,210,1482553445726908420,2022-01-16 03:21:00+00:00,KatieHowse2,Wordle 210 3/6\n\n⬛⬛🟨⬛⬛\n🟩🟩🟩⬛⬛\n🟩🟩🟩🟩🟩
